In [1]:
import pandas as pd

In [53]:
df = pd.read_csv('AirBnB_NY/AB_NYC_2019.csv')

In [54]:
# Use the pivot_table function to create a detailed summary that reveals the
# average price for different combinations of neighbourhood_group and
# room_type. This analysis will help identify high-demand areas and optimize
# pricing strategies across various types of accommodations (e.g., Entire
# home/apt vs. Private room).


pivot_table = pd.pivot_table(
  df,
  values='price',                              # The data to aggregate
  index='neighbourhood_group',                 # Rows will be neighbourhood groups
  columns='room_type',                         # Columns will be room types
  aggfunc='mean'                               # Aggregate using mean to get the average price
)

print(pivot_table)

room_type            Entire home/apt  Private room  Shared room
neighbourhood_group                                            
Bronx                     127.506596     66.788344    59.800000
Brooklyn                  178.327545     76.500099    50.527845
Manhattan                 249.239109    116.776622    88.977083
Queens                    147.050573     71.762456    69.020202
Staten Island             173.846591     62.292553    57.444444


In [55]:
# Transform the dataset from a wide format to a long format using the melt
# function. 
# This restructuring facilitates more flexible and detailed analysis of
# key metrics like price and minimum_nights, enabling the identification of
# trends, outliers, and correlations.

df_melted = pd.melt(
    df,
    id_vars=['neighbourhood_group'],          # Columns to keep as identifiers
    value_vars=['price', 'minimum_nights'],   # Columns to unpivot
    var_name='metric',
    value_name='value'
)

display(df_melted)

,neighbourhood_group,metric,value
0,Brooklyn,price,149
1,Manhattan,price,225
2,Manhattan,price,150
3,Brooklyn,price,89
4,Manhattan,price,80
...,...,...,...
97785,Brooklyn,minimum_nights,2
97786,Brooklyn,minimum_nights,4
97787,Manhattan,minimum_nights,10
97788,Manhattan,minimum_nights,1


In [56]:
#Create a new column availability_status using the apply function, classifying each listing into one of three 
#categories based on the availability_365 column:
# ▪ "Rarely Available": Listings with fewer than 50 days of availability in a year.
# ▪ "Occasionally Available": Listings with availability between 50 and 200 days.
# ▪ "Highly Available": Listings with more than 200 days of availability.

#df[['neighbourhood_group', 'availability_365']]

def availability_status(availability_365):
    if availability_365 < 50:
        return "Rarely Available"
    elif availability_365 >= 50 and availability_365 < 200:
        return "Occasionally Available"
    else:
        return "Highly Available"

df['availability_status'] = df['availability_365'].apply(availability_status)


In [57]:
# Analyze trends and patterns using the new availability_status column, and
# investigate potential correlations between availability and other key
# variables like price, number_of_reviews, and neighbourhood_group to
# uncover insights that could inform marketing and operational strategies.

In [58]:
corr_on_price = df['availability_365'].corr(df['price'])
print('Correlations between availability and price')
print(corr_on_price)

print('')

corr_on_num_of_rev = df['availability_365'].corr(df['number_of_reviews'])
print('Correlations between availability and number_of_reviews')
print(corr_on_num_of_rev)

Correlations between availability and price
0.08182882742168796

Correlations between availability and number_of_reviews
0.1720275814629295


In [59]:
# Neighbourhood_group and availability_status
availability_by_neighbourhood = df.groupby(['neighbourhood_group', 'availability_status']).size()

print("Availability by Neighbourhood Group:")
print(availability_by_neighbourhood)

Availability by Neighbourhood Group:
neighbourhood_group  availability_status   
Bronx                Highly Available            416
                     Occasionally Available      369
                     Rarely Available            306
Brooklyn             Highly Available           4781
                     Occasionally Available     4290
                     Rarely Available          11033
Manhattan            Highly Available           6055
                     Occasionally Available     4258
                     Rarely Available          11348
Queens               Highly Available           1923
                     Occasionally Available     1665
                     Rarely Available           2078
Staten Island        Highly Available            193
                     Occasionally Available      107
                     Rarely Available             73
dtype: int64


In [60]:
# Calculate average price by availability_status
average_price_by_availability = df.groupby('availability_status')['price'].mean()

print("Avg price by availability_status:")
print(average_price_by_availability)

Avg price by availability_status:
availability_status
Highly Available          180.308124
Occasionally Available    155.093461
Rarely Available          136.851800
Name: price, dtype: float64


In [61]:
# Calculate average number of reviews by availability_status
average_reviews_by_availability = df.groupby('availability_status')['number_of_reviews'].mean()

print("Avg number_of_reviews by availability_status:")
print(average_reviews_by_availability)

Avg number_of_reviews by availability_status:
availability_status
Highly Available          34.596948
Occasionally Available    31.881186
Rarely Available          13.476729
Name: number_of_reviews, dtype: float64


In [62]:
# Perform basic descriptive statistics (e.g., mean, median, standard deviation) on
# numeric columns such as price, minimum_nights, and number_of_reviews to
# summarize the dataset's central tendencies and variability, which is crucial for
# understanding overall market dynamics.

summary_statistics = df[['price', 'minimum_nights', 'number_of_reviews']].describe()

print("Basic Descriptive Statistics:")
print(summary_statistics)

Basic Descriptive Statistics:
              price  minimum_nights  number_of_reviews
count  48895.000000    48895.000000       48895.000000
mean     152.720687        7.029962          23.274466
std      240.154170       20.510550          44.550582
min        0.000000        1.000000           0.000000
25%       69.000000        1.000000           1.000000
50%      106.000000        3.000000           5.000000
75%      175.000000        5.000000          24.000000
max    10000.000000     1250.000000         629.000000


In [ ]:
# Convert the last_review column to a datetime object and set it as the index of the DataFrame 
# to facilitate time-based analyses.

In [63]:
df['last_review'] = pd.to_datetime(df['last_review'])
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,availability_status
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,Highly Available
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,Highly Available
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaT,NaN,1,365,Highly Available
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,Occasionally Available
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,Rarely Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaT,NaN,2,9,Rarely Available
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaT,NaN,2,36,Rarely Available
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaT,NaN,1,27,Rarely Available
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaT,NaN,6,2,Rarely Available


In [65]:
df.set_index('last_review', inplace=True)
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,availability_status
last_review,,,,,,,,,,,,,,,,
2018-10-19,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.21,6,365,Highly Available
2019-05-21,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355,Highly Available
NaT,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,1,365,Highly Available
2019-07-05,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194,Occasionally Available
2018-11-19,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0,Rarely Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaT,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,2,9,Rarely Available
NaT,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,2,36,Rarely Available
NaT,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,1,27,Rarely Available


In [66]:
# Resample the data to observe monthly trends in the number of reviews and
# average prices, providing insights into how demand and pricing fluctuate
# over time.
monthly_reviews = df['number_of_reviews'].resample('M').sum()

print("Monthly number_of_reviews:")
print(monthly_reviews)

Monthly Number of Reviews:
last_review
2011-03-31         1
2011-04-30        14
2011-05-31         2
2011-06-30         0
2011-07-31         0
               ...  
2019-03-31     11307
2019-04-30     24828
2019-05-31     86486
2019-06-30    619907
2019-07-31    243262
Name: number_of_reviews, Length: 101, dtype: int64


In [67]:
monthly_average_price = df['price'].resample('M').mean()

print("Monthly avg price:")
print(monthly_average_price)

Monthly Average Price:
last_review
2011-03-31     55.000000
2011-04-30    250.000000
2011-05-31    249.000000
2011-06-30           NaN
2011-07-31           NaN
                 ...    
2019-03-31    136.243902
2019-04-30    143.211538
2019-05-31    139.526373
2019-06-30    141.613443
2019-07-31    144.487831
Name: price, Length: 101, dtype: float64


In [70]:
# Group the data by month to calculate monthly averages and analyze
# seasonal patterns, enabling better forecasting and strategic planning around
# peak periods.

df['month'] = df.index.month

In [76]:
monthly_price_avg = df.groupby('month')[['price', 'number_of_reviews', 'minimum_nights']].mean()

print("Monthly price avg:")
print(monthly_price_avg)

Monthly price avg:
            price  number_of_reviews  minimum_nights
month                                               
1.0    150.478904          11.195456        7.467364
2.0    155.716883          12.301299        8.876623
3.0    136.836957          11.670807        9.510870
4.0    139.504979          14.503556        8.012328
5.0    137.909594          20.181025        7.369921
6.0    140.769667          46.255795        4.109648
7.0    142.290382          42.700354        3.668183
8.0    132.133173           7.697660        8.588482
9.0    149.494463           9.278827        7.157655
10.0   142.065330           9.811125        7.376455
11.0   138.532815          10.576857        8.443869
12.0   160.796610          11.538983        7.054802
